# 2-3,自动微分机制

神经网络通常依赖反向传播求梯度来更新网络参数，求梯度过程通常是一件非常复杂而容易出错的事情。

而深度学习框架可以帮助我们自动地完成这种求梯度运算。

`Tensorflow` 一般使用梯度磁带 `tf.GradientTape` 来记录正向运算过程，然后反播磁带自动得到梯度值。

这种利用 `tf.GradientTape` 求微分的方法叫做 `Tensorflow` 的自动微分机制。

### 一，利用梯度磁带求导数

In [4]:
import tensorflow as tf
import numpy as np
# import os

# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# f(x) = a*x^2 + b*x + c

x = tf.Variable(0.0, name="x", dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as tape:
    y = a * tf.pow(x, 2) + b * x + c

dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(-2.0, shape=(), dtype=float32)


In [6]:
# 对常量也可以进行求导，需要利用 watch 方法

with tf.GradientTape() as tape:
    tape.watch([a, b, c])
    y = a * tf.pow(x, 2) + b * x + c

dy_dx, dy_da, dy_db, dy_dc = tape.gradient(y, [x, a, b, c])
print(dy_dx, dy_da, dy_db, dy_dc, sep="\n")

tf.Tensor(-2.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [7]:
# 求二阶导数
with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:
        y = a * tf.pow(x, 2) + b * x + c
    dy_dx = tape1.gradient(y, x)
dy2_dx2 = tape2.gradient(dy_dx, x)
print(dy2_dx2)

tf.Tensor(2.0, shape=(), dtype=float32)
